In [1]:
from paddlenlp import Taskflow

In [2]:
tag = Taskflow("pos_tagging",user_dict="user_dict.txt")

In [3]:
data = tag("山东省日照市秦楼街道城建花园社区济南路壹号学府1号楼-1-1201")
print(data)

[('山东省', 'LOC'), ('日照市', 'LOC'), ('秦楼街道城建花园社区', 'LOC'), ('济南路', 'LOC'), ('壹号学府', 'ADDRESS'), ('1号楼', 'LOC'), ('-', 'w'), ('1', 'm'), ('-', 'w'), ('1201', 'm')]


数据解析处理思路:

识别出来小区的信息数据。然后将其前面的数据处理掉:


In [4]:
dict = []
str = []
for item in data:
    dict.append(item[1])
    str.append(item[0])
print(dict)

['LOC', 'LOC', 'LOC', 'LOC', 'ADDRESS', 'LOC', 'w', 'm', 'w', 'm']


In [12]:
tag = 'ADDRESS'
tagIndex = -1
list = []
data = []
for i,item in enumerate(dict):
    if item == tag:
        tagIndex = i
    if tagIndex!=-1:
        list.append(item)
        data.append(str[i])

print(list)

['ADDRESS', 'LOC', 'w', 'm', 'w', 'm']


In [21]:

import sys
result = []
with open('matchers.txt','r') as f:
    for line in f:
        result.append(line.strip('\n'))
print(result)


['ADDRESS,LOC,w,m,w,m=>m_1,m_2,,,']


In [22]:
def match(match,list):
    str = ",".join(list)
    array =  match.split("=>")
    m = array[0]
    return m==str,array[1];

print(match(result[0],list)) 
# print(match('ADDRESS,LOC,w,m,w,m,m2',list) 

(True, 'm_1,m_2,,,')


In [71]:
def parser(matcher,list,data):
    
    res = []
   
    for i,m in enumerate(matcher.split(",")):
        flags = m.split("_");
        val = "null"
        index = 1
        if len(flags) == 2 :
            f = flags[0]
            n = int(flags[1])           
            for j,item in enumerate(list):    
                if item == f:                   
                    if index == n:
                        val = data[j]                                    
                        res.append(val)
                    index = index +1
        else:
             res.append(val)
    return res 

for item in result:
    flag,matcher = match(item,list)
    if flag:
       res =  parser(matcher,list,data)
       print(res) 


['1', '1201', 'null', 'null', 'null']
